<a href="https://colab.research.google.com/github/btg1631/study_data_analytics/blob/main/docs/quests/visuallizations/EDAs/multivariate_OrdersDeliveryList.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [2]:
df_SMDWD = pd.read_csv("ShoppingMallDeliveryWithDate.csv")
df_SMDWD[:2]

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077.0,1,PC쇼핑몰,20181227
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42.0,2,모바일웹,20181229


In [3]:
df_SMDWD.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46749 entries, 0 to 46748
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   결제수단    46749 non-null  object 
 1   배송번호    46749 non-null  object 
 2   배송시작일   46749 non-null  object 
 3   배송완료일   46749 non-null  object 
 4   상품구매금액  46749 non-null  int64  
 5   상품번호    46661 non-null  float64
 6   수량      46749 non-null  int64  
 7   주문경로    46749 non-null  object 
 8   주문일     46749 non-null  int64  
dtypes: float64(1), int64(3), object(5)
memory usage: 3.2+ MB


In [35]:
# 1. 상품구매금액이 80,000원 이상인 주문들을 찾으세요.
df_SMDWD.loc[df_SMDWD['상품구매금액'] >= 80000][:3]

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송시작일_datetime,배송완료일_datetime,배송기간
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42,2,모바일웹,20181229,2019-01-03,2019-01-07,4
3,무통장입금,D-20181231-0000087-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,81600,1085,1,모바일웹,20181231,2019-01-03,2019-01-07,4
4,무통장입금,D-20181231-0000108-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,94710,1106,1,모바일웹,20181231,2019-01-07,2019-01-11,4


In [36]:
# 2. 모바일웹을 통해 주문된 상품 중, 상품구매금액이 평균 이상인 주문들을 찾으세요.
order_mobile = "주문경로 == '모바일웹'"
mean = df_SMDWD['상품구매금액'].mean()
price_mean = f"상품구매금액 >= {mean}"  # mean = 69368
result = df_SMDWD.query(price_mean)
result[["주문경로", "상품구매금액"]][:3]

,주문경로,상품구매금액
0,PC쇼핑몰,71450
1,모바일웹,141240
3,모바일웹,81600


In [33]:
# 3. 2019년 1월 7일 이전에 배송이 시작된 주문들을 찾으세요.
delivery_start = '배송시작일 <= "2019-01-07"'
df_SMDWD[["배송번호", "배송시작일"]].query(delivery_start)[:3]

,배송번호,배송시작일
1,D-20181229-0000119-00,2019-01-03 오전 9:30
2,D-20181230-0000100-00,2019-01-03 오전 9:30
3,D-20181231-0000087-00,2019-01-03 오전 9:30


In [8]:
# 4. 배송 시작일과 배송 완료일이 3일 이상 차이 나는 주문들을 찾으세요.
def convert_time(x):
    date, ampm, time = x.split()
    if ampm == "오후" and time != "12:00":
        hour, minute = map(int, time.split(":"))
        time = f"{hour+12 if hour != 12 else hour}:{minute}"
    return pd.to_datetime(f"{date}", format="%Y-%m-%d")

df_SMDWD['배송시작일_datetime'] = df_SMDWD['배송시작일'].apply(convert_time)
df_SMDWD['배송완료일_datetime'] = df_SMDWD['배송완료일'].apply(convert_time)

In [32]:
df_SMDWD["배송기간"] = df_SMDWD["배송완료일_datetime"] - df_SMDWD["배송시작일_datetime"]
df_SMDWD["배송기간"] = df_SMDWD["배송기간"].dt.days
result = '배송기간 >= 3'
result = df_SMDWD.query(result)
result[["배송기간", "배송완료일_datetime", "배송시작일_datetime"]][:3]

,배송기간,배송완료일_datetime,배송시작일_datetime
0,4,2019-01-11,2019-01-07
1,4,2019-01-07,2019-01-03
2,4,2019-01-07,2019-01-03


In [31]:
# 5. 무통장입금을 결제수단으로 사용한 주문 중, 상품번호가 1000번 이상인 주문들을 찾으세요.
bank = '결제수단 == "무통장입금"'
product_number = '상품번호 >= 1000'
result = f'{bank} and {product_number}'
result = df_SMDWD.query(result)
result[["결제수단", "상품번호"]][:3]

,결제수단,상품번호
0,무통장입금,1077
2,무통장입금,1271
3,무통장입금,1085


In [30]:
# 6. 상품번호가 50번 이하이거나 1000번 이상인 주문들을 찾으세요.
df_SMDWD['상품번호'] = df_SMDWD['상품번호'].fillna(0).astype(int)
product_number_more = df_SMDWD['상품번호'] <= 50
product_number_less = df_SMDWD['상품번호'] >= 1000
result = df_SMDWD[product_number_more | product_number_less]
result[:3]

,결제수단,배송번호,배송시작일,배송완료일,상품구매금액,상품번호,수량,주문경로,주문일,배송시작일_datetime,배송완료일_datetime,배송기간
0,무통장입금,D-20181227-0000648-00,2019-01-07 오전 8:56,2019-01-11 오전 5:32,71450,1077,1,PC쇼핑몰,20181227,2019-01-07,2019-01-11,4
1,무통장입금,D-20181229-0000119-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,141240,42,2,모바일웹,20181229,2019-01-03,2019-01-07,4
2,무통장입금,D-20181230-0000100-00,2019-01-03 오전 9:30,2019-01-07 오전 5:32,13910,1271,1,모바일웹,20181230,2019-01-03,2019-01-07,4


In [26]:
# 7. 주문일이 20181231인 주문 중, 수량이 1개 이상인 주문들을 찾으세요.
order_date = '주문일 == 20181231'
quantity = '수량 == 1'
result = f'{order_date} and {quantity}'
result = df_SMDWD.query(result)
result[["주문일", "수량"]][:3]

,주문일,수량
3,20181231,1
4,20181231,1
5,20181231,1


In [25]:
# 8. PC쇼핑몰을 통해 주문된 상품 중, 배송번호가 'D-20181231'로 시작하는 주문들을 찾으세요.
order_pc = df_SMDWD["주문경로"] == "PC쇼핑몰"
delivery_num = df_SMDWD["배송번호"].str.contains('D-20181231')
result = order_pc & delivery_num
result = df_SMDWD.loc[result]
result[["주문경로", "배송번호"]][:3]

,주문경로,배송번호
6,PC쇼핑몰,D-20181231-0000184-00
7,PC쇼핑몰,D-20181231-0000184-01
8,PC쇼핑몰,D-20181231-0000184-02


In [23]:
# 9. 상품구매금액이 70,000원 이상 100,000원 이하인 주문들 중, 수량이 1개인 주문들을 찾으세요.
price1 = '상품구매금액 >= 70000'
price2 = '상품구매금액 <= 100000'
price = f'{price1} and {price2}'
num = '수량 == 1'
result = f'{price} and {num}'
result = df_SMDWD.query(result)
result[["상품구매금액", "수량"]][:3]

,상품구매금액,수량
0,71450,1
3,81600,1
4,94710,1


In [40]:
# 10. 모바일웹을 통한 주문 중, 배송번호가 'D-20181230' 이후로 시작되고 상품구매금액이 50,000원 이상인 주문들을 찾으세요.
order_pc = "주문경로 == '모바일웹'"
delivery_num = '주문일 >= 20181230'
price = '상품구매금액 >= 50000'
result = f'{order_pc} and {delivery_num} and {price}'
result = df_SMDWD.query(result)
result[["주문경로", "배송번호", "상품구매금액", "주문일"]][:3]

,주문경로,배송번호,상품구매금액,주문일
3,모바일웹,D-20181231-0000087-00,81600,20181231
4,모바일웹,D-20181231-0000108-00,94710,20181231
5,모바일웹,D-20181231-0000108-01,75240,20181231
